In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import Polygon
import pandas as pd
import osmnx as ox
from shapely.ops import split
from shapely.geometry import MultiPolygon, Polygon
from shapely.geometry import LineString
import numpy as np
from src.preprocess import *

from src.download_roads_data import get_roads_from_osmnx

In [2]:
pictures = gpd.read_file("data/S2_GRID.geojson")

noise = gpd.read_file("data/NOISE.geojson")

transport_lines = gpd.read_file("data/TRANSPORT_LINES.geojson")

noise_barriers_1 = gpd.read_file('data/NOISE_BARRIERS.geojson')
noise_barriers_2 = gpd.read_file('data/NOISE_BARRIERS_2.geojson')
noise_barriers_1.drop(columns='ID_CLONA', inplace=True)
noise_barriers_2.drop(columns='ID_VAL', inplace=True)
noise_barriers = pd.concat([noise_barriers_1, noise_barriers_2]).reset_index(drop = True)

parks = gpd.read_file('data/PARKS.geojson')

buildings = gpd.read_file('data/BUILDINGS.geojson')

population = gpd.read_file('data/DEMOGRAPHY.geojson')

roads = get_roads_from_osmnx()

In [3]:
noise_bounds = noise.unary_union

transport_lines = gpd.clip(transport_lines, noise_bounds)
roads = gpd.clip(roads, noise_bounds)
noise_barriers = gpd.clip(noise_barriers, noise_bounds)

/tmp/ipykernel_49577/1638377482.py:1: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  noise_bounds = noise.unary_union


In [4]:
roads

,,,osmid,highway,lanes,name,oneway,reversed,length,geometry,maxspeed,tunnel,ref,access,bridge,junction,width,area
u,v,key,,,,,,,,,,,,,,,,
903956998,903957028,1,"[76707203, 76707204]",residential,NaN,Strnady,False,False,303.013107,"LINESTRING (-746945.416 -1058496.782, -746955....",40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
903957028,903956998,1,"[76707203, 76707204]",residential,NaN,Strnady,False,True,303.013107,"LINESTRING (-747033.393 -1058479.612, -747073....",40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
903956899,903956962,0,1272026408,residential,NaN,Strnady,False,False,178.562104,"LINESTRING (-747113.567 -1058529.735, -747099....",40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
903956962,903956899,0,1272026408,residential,NaN,Strnady,False,True,178.562104,"LINESTRING (-747049.617 -1058363.137, -747054....",40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
903956879,903957028,0,76707204,residential,NaN,Strnady,False,True,120.471328,"LINESTRING (-747006.758 -1058364.83, -747004.0...",40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261886987,1261887227,0,110515784,residential,2,Petráčkova,False,True,70.702019,"LINESTRING (-737255.536 -1034500.78, -737264.3...",30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1261886906,1261887154,0,110515759,living_street,NaN,Petráčkova,False,False,11.078879,"LINESTRING (-737293.444 -1034496.381, -737294....",40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7187797477,7187797476,0,769953096,living_street,NaN,Mikanových,False,True,29.700207,"LINESTRING (-737833.745 -1034515.353, -737837....",20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# from src.s2_grid import split_polygon
from src.aggregation import *

In [6]:
for i, picture in enumerate(pictures['geometry']):
    noise_for_picture = gpd.clip(noise, picture)
    roads_for_picture = gpd.clip(roads, picture)
    transport_lines_for_picture = gpd.clip(transport_lines, picture)
    buildings_for_picture = gpd.clip(buildings, picture)
    bariers_for_picture = gpd.clip(noise_barriers, picture)
    parks_for_picture = gpd.clip(parks, picture)
    
    population_subgdf = gpd.GeoDataFrame(geometry=[picture], crs = population.crs)
    population_for_picture = gpd.sjoin(population, population_subgdf, how='inner', predicate='intersects')
    population_for_picture = population_for_picture.drop(columns='index_right')

    
    pixels = create_pixels_gdf(picture)

    #liczenie wzonego halasu dla kazdego piksela
    pixels["weighted_db_hi"] = pixels.geometry.apply(lambda pixel: calculate_weighted_db(pixel, noise_for_picture))
    # pixels["weighted_db_hi"] = pixels["weighted_db_hi"].apply(round_to_nearest_5)
    #droga z max predkoscia na pixeulu
    pixels["max_speed"] = pixels["geometry"].apply(lambda pixel: get_max_speed(pixel, roads_for_picture))
    
    #czy pixel posiada transport line
    pixels['has_transport_line'] = pixels.geometry.apply(lambda pixel: transport_lines_for_picture.geometry.intersects(pixel).any())

    #noise barriers
    pixels['has_barrier'] = pixels.geometry.apply(lambda pixel: get_barrier(pixel, bariers_for_picture))

    #parks
    pixels['parks_area'] = pixels.geometry.apply(lambda pixel: get_parks_area(pixel, parks_for_picture))

    #buildings
    pixels['buildings_levels'] = pixels.geometry.apply(lambda pixel: get_buildings_levels(pixel, buildings_for_picture))

    #population
    pixels['population'] = pixels.geometry.apply(lambda pixel: get_population(pixel, population_for_picture))
    
    #pixels.drop(columns="geometry").to_csv(f"data/pictures/picture_{i}.csv", index=False)
    pixels.to_file(f"data/pictures/picture_{i}.geojson", driver="GeoJSON")

KeyboardInterrupt: 